In [ ]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from ast import literal_eval
import heapq
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
CF  = pd.read_csv('CareersFutureCleaned.csv')

In [ ]:
CF

In [ ]:
CF['clean_jd'] = CF.apply(lambda x: str(x['Job Description']), axis=1)
CF['clean_jd'] = CF['clean_jd'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
CF['clean_jd'] = CF["clean_jd"].fillna('').apply(lambda x: x.lower())
CF['Skills'] = CF['Skills'].apply(lambda x: eval(x))
CF['Skills'] = CF["Skills"].apply(lambda a: [x.lower() for x in a])
CF['Industry'] = CF['Industry'].apply(lambda x: eval(x))

In [ ]:
CF_jd_skills = CF[['clean_jd', 'Industry', 'Job Title', 'Skills']]

In [ ]:
SFW  = pd.read_csv('sfw_sector.csv')

In [ ]:
SFW

In [ ]:
SFW['Skill Title'][0]

In [ ]:
SFW_new = pd.read_csv('sf_model_final.csv')

In [ ]:
SFW_new

In [ ]:
SFW_new2 = SFW_new.groupby(['Job_Role_Replaced', 'Sector']).agg({'Job Role': lambda series: list(series), 'Skill description': lambda series: list(series), 'Skill Title':lambda series: list(series)})

In [ ]:
SFW_new2.head()

In [ ]:
SFW_new2.reset_index(inplace=True)

In [ ]:
SFW_new2

In [ ]:
SFW_new2.to_csv('final_new_sfw.csv')

In [ ]:
def to_lower(x):
    y = []
    for i in range(len(x)):
        y.append(x[i].lower())
    return y

In [ ]:
SFW_new2['Skill Title'] = SFW_new2['Skill Title'].apply(lambda x: to_lower(x))

In [ ]:
skills_lib  = pd.read_csv('tableau no jobstreet 150321.csv')

In [ ]:
skills_lib

In [ ]:
skills_cf = skills_lib

In [ ]:
skills_cf_new = skills_cf.sort_values('Score', ascending=False).drop_duplicates(['MyCFW_Skill'])

In [ ]:
skills_cf_new

In [ ]:
CF_jd_skills

In [ ]:
mydict = dict(zip(skills_cf_new['MyCFW_Skill'], skills_cf_new['Skill Title V2'])) # key is cf skills, value is sfw skills

In [ ]:
mydict

In [ ]:
def checker(x):
    new_list = []
    for i in range(0, len(x)):
        if x[i] in mydict.keys():
            new_list.append(mydict.get(x[i]))
        else:
            new_list.append(x[i])
    return new_list

In [ ]:
CF_jd_skills['skills_replaced'] = CF_jd_skills['Skills'].apply(lambda x: checker(x))

In [ ]:
CF_jd_skills

In [ ]:
#only 1 industry
CF_jd_skills_1_ind = CF_jd_skills.loc[np.array(list(map(len, CF_jd_skills['Industry'].values)))==1]                                 

In [ ]:
CF_jd_skills_1_ind['Industry_nolist'] = CF_jd_skills_1_ind['Industry'].apply(lambda x: ''.join(x))

In [ ]:
CF_jd_skills_1_ind

In [ ]:
CF_jd_skills_1_ind.to_csv('single_ind_cf.csv')

In [ ]:
CF_jd_skills_1_ind = pd.read_csv('single_ind_cf.csv')
CF_jd_skills_1_ind

In [ ]:
SFW = pd.read_csv('final_new_sfw.csv')
SFW

# Infocomm Tech -> IT

In [ ]:
selection = ['Information Technology']

IT = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in selection if item in x))

IT = CF_jd_skills_1_ind[IT]

In [ ]:
len(IT)

In [ ]:
IT.reset_index(inplace=True)

In [ ]:
sfw_sector_IT = SFW[SFW['Sector'] == "Infocomm Technology"]

In [ ]:
sfw_sector_IT.reset_index(inplace=True)

In [ ]:
sfw_sector_IT

In [ ]:
#skill and job title matching using job description
design_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []
#cf = nlp(str(CF_jd_skills_IT.iloc[1]['skills_replaced']))
for i in range(0, len(IT)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(IT.iloc[i]['skills_replaced']))
    for j in range(0, len(sfw_sector_IT)):
        sfw = nlp(str(sfw_sector_IT['Skill Title'][j]))
        #append items
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][j])
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
len(scores_main[72])

In [ ]:
len(scores_main)

In [ ]:
IT.iloc[1]

In [ ]:
import heapq
ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores)), key=scores.__getitem__)

In [ ]:
sfw_sector_IT.iloc[ind1]

In [ ]:
sfw_sector_IT.iloc[ind2]

In [ ]:
sfw_sector_IT.iloc[ind3]

In [ ]:
closest_1_sfw_jobs = []
closest_2_sfw_jobs = []
closest_3_sfw_jobs = []
cf_jobs = []
for i in range(73):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind1])
    closest_2_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind2])
    closest_3_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind3])
    cf_jobs.append(IT['Job Title'][i])

In [ ]:
df_IT_72 = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_3_sfw_jobs':closest_3_sfw_jobs})

In [ ]:
df_IT_72.head(50)

In [ ]:
CF_jd_skills_1_ind['Industry_nolist'].unique()

In [ ]:
skills_cf_new['Sector'].unique()

# Fin Serv

In [ ]:
fin_serv = [['Banking and Finance'], ['Insurance'], ['Consulting'], ['Risk Management']]
fin_serv_dataframes = []

for i in fin_serv:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    fin_serv_dataframes.append(df)

fin_serv_df = pd.concat(fin_serv_dataframes)
fin_serv_df.reset_index(inplace=True)

sfw_sector_fin_serv = SFW[SFW['Sector'] == "Financial Services"]
sfw_sector_fin_serv.reset_index(inplace=True)

In [ ]:
fin_serv_df

In [ ]:
print(len(fin_serv_df))
print(len(sfw_sector_fin_serv))

In [ ]:
fin_serv_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(fin_serv_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(fin_serv_df.iloc[i]['skills_replaced']))
    for j in range(0, len(sfw_sector_fin_serv)):
        sfw = nlp(sfw_sector_fin_serv['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_fin_serv['Job_Role_Replaced'][j])
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_fin_serv['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_fin_serv['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_fin_serv['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[ind3])
    cf_jobs.append(fin_serv_df['Job Title'][i])

df_fin_serv = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_fin_serv.head(50)

# Acc

In [ ]:
acc = [['Accounting / Auditing / Taxation']]
acc_dataframes = []

for i in acc:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    acc_dataframes.append(df)

acc_df = pd.concat(acc_dataframes)
acc_df.reset_index(inplace=True)

sfw_sector_acc = SFW[SFW['Sector'] == "Accountancy"]
sfw_sector_acc.reset_index(inplace=True)

In [ ]:
print(len(acc_df))
print(len(sfw_sector_acc))

In [ ]:
acc_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(acc_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(acc_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_acc)):
        sfw = nlp(str(sfw_sector_acc['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
len(scores_main)

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(acc_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
df_acc.head(50)

In [ ]:
df_acc.to_csv('accounting.csv')

# IT

In [ ]:
#mapped to infocomm tech
IT = [['Information Technology']]
IT_dataframes = []

for i in IT:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    IT_dataframes.append(df)

IT_df = pd.concat(IT_dataframes)
IT_df.reset_index(inplace=True)

sfw_sector_IT = SFW[SFW['Sector'] == "Infocomm Technology"]
sfw_sector_IT.reset_index(inplace=True)

print(len(IT_df))
print(len(sfw_sector_IT))

In [ ]:
IT_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(IT_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(IT_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_IT)):
        sfw = nlp(sfw_sector_IT['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_IT['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(IT_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
df_acc.head(50)

# Design

In [ ]:
#mapped to design
des = [['Architecture / Interior Design'], ['Design']]
des_dataframes = []

for i in IT:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    des_dataframes.append(df)

design_df = pd.concat(des_dataframes)

sfw_sector_des = SFW[SFW['Sector'] == "Design"]
sfw_sector_des.reset_index(inplace=True)

print(len(design_df))
print(len(sfw_sector_des))

In [ ]:
des_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(design_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(design_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_des)):
        sfw = nlp(sfw_sector_des['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_des['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_des['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_des['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_des['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(design_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_2_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_3_score})
df_acc.head(50)

# Education

In [ ]:
edu = [['Education and Training']]
edu_dataframes = []

for i in edu:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    edu_dataframes.append(df)

edu_df = pd.concat(edu_dataframes)
edu_df.reset_index(inplace=True)

sfw_sector_edu = SFW[SFW['Sector'] == "Training and Adult Education"]
sfw_sector_edu.reset_index(inplace=True)

print(len(edu_df))
print(len(sfw_sector_edu))

In [ ]:
edu_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(edu_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(edu_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_edu)):
        sfw = nlp(str(sfw_sector_edu['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_edu['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_edu['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_edu['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_edu['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(edu_df['Job Title'][i])

df_edu = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_edu.head(50)

In [ ]:
df_edu.to_csv('education.csv')

# Engine

In [ ]:
engine = [['Engineering']]
engine_dataframes = []

for i in engine:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    engine_dataframes.append(df)

engine_df = pd.concat(engine_dataframes)
engine_df.reset_index(inplace=True)

sfw_sector_engine = SFW[SFW['Sector'] == "Engineering Services"]
sfw_sector_engine.reset_index(inplace=True)

print(len(engine_df))
print(len(sfw_sector_engine))

In [ ]:
engine_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(engine_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(engine_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_engine)):
        sfw = nlp(sfw_sector_engine['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_engine['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[ind3])
    cf_jobs.append(acc_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_acc.head(50)

# Tourism

In [ ]:
trav = [['Travel / Tourism']]
trav_dataframes = []

for i in trav:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    trav_dataframes.append(df)

travel_df = pd.concat(trav_dataframes)
travel_df.reset_index(inplace=True)

sfw_sector_trav = SFW[SFW['Sector'] == "Tourism"]
sfw_sector_trav.reset_index(inplace=True)

print(len(travel_df))
print(len(sfw_sector_trav))

In [ ]:
travel_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(travel_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(travel_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_trav)):
        sfw = nlp(str(sfw_sector_trav['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_trav['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_trav['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_trav['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_trav['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(travel_df['Job Title'][i])

df_tourism = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_tourism.head(50)

In [ ]:
df_tourism.to_csv('tourism.csv')

# Media

In [ ]:
adv = [['Advertising / Media'], ['Entertainment']]
adv_dataframes = []

for i in adv:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    adv_dataframes.append(df)

media_df = pd.concat(adv_dataframes)
media_df.reset_index(inplace=True)

sfw_sector_media = SFW[SFW['Sector'] == "Media"]
sfw_sector_media.reset_index(inplace=True)

print(len(media_df))
print(len(sfw_sector_media))

In [ ]:
media_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(media_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(media_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_media)):
        sfw = nlp(str(sfw_sector_media['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_media['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
media_df

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_media['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_media['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_media['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(media_df['Job Title'][i])

df_media = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_media.head(50)

In [ ]:
df_media.to_csv('media.csv')

# Healthcare

In [ ]:
health = [['Healthcare / Pharmaceutical'], ['Personal Care / Beauty'], ['Medical / Therapy Services']]
health_dataframes = []

for i in health:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    health_dataframes.append(df)

health_df = pd.concat(health_dataframes)
health_df.reset_index(inplace=True)

sfw_sector_health = SFW[SFW['Sector'] == "Healthcare"]
sfw_sector_health.reset_index(inplace=True)

print(len(health_df))
print(len(sfw_sector_health))

In [ ]:
health_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(health_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(health_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_health)):
        sfw = nlp(sfw_sector_health['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_health['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_health['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_health['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_health['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(health_df['Job Title'][i])

df_health = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_health.head(50)

In [ ]:
df_health.to_csv('health.csv')

# Retail

In [ ]:
retail = [['Sales / Retail']]
retail_dataframes = []

for i in retail:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    retail_dataframes.append(df)

retail_df = pd.concat(retail_dataframes)
retail_df.reset_index(inplace=True)

sfw_sector_retail = SFW[SFW['Sector'] == "Retail"]
sfw_sector_retail.reset_index(inplace=True)

print(len(retail_df))
print(len(sfw_sector_retail))

In [ ]:
retail_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(retail_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(retail_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_retail)):
        sfw = nlp(str(sfw_sector_retail['Skill Title'][j]))
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_retail['Job Role'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_retail['Job Role'][ind1])
    closest_1_score.append(scores_main[i][ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_retail['Job Role'][ind2])
    closest_2_score.append(scores_main[i][ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_retail['Job Role'][ind3])
    closest_3_score.append(scores_main[i][ind3])
    cf_jobs.append(retail_df['Job Title'][i])

df_retail = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_retail.head(50)

In [ ]:
df_retail.to_csv('retail.csv')

# Logistics

In [ ]:
log = [['Logistics / Supply Chain']]
log_dataframes = []

for i in log:
    df = CF_jd_skills_1_ind.Industry.apply(lambda x: any(item for item in i if item in x))
    df = CF_jd_skills_1_ind[df]
    df.reset_index(inplace=True)
    log_dataframes.append(df)

logistics_df = pd.concat(log_dataframes)
logistics_df.reset_index(inplace=True)

sfw_sector_log = SFW[SFW['Sector'] == "Logistics"]
sfw_sector_log.reset_index(inplace=True)

print(len(logistics_df))
print(len(sfw_sector_log))

In [ ]:
logs_sim = {'cf job titles': [], 'sfw job role': [], 'sim score': []}

scores_main = []
sfw_jobs_main = []

for i in range(0, len(logistics_df)):
    scores = []
    sfw_jobs = []
    cf = nlp(str(logistics_df.iloc[i]['skills_replaced']))
    
    for j in range(0, len(sfw_sector_log)):
        sfw = nlp(sfw_sector_log['Skill Title'][j])
        scores.append(sfw.similarity(cf))
        sfw_jobs.append(sfw_sector_log['Job_Role_Replaced'][j])
        
    scores_main.append(scores)
    sfw_jobs_main.append(sfw_jobs)
    print("done cf job role: " + str(i))

In [ ]:
closest_1_sfw_jobs = []
closest_1_score = []

closest_2_sfw_jobs = []
closest_2_score = []

closest_3_sfw_jobs = []
closest_3_score = []
cf_jobs = []

for i in range(len(scores_main)):
    ind1, ind2, ind3 = heapq.nlargest(3, range(len(scores_main[i])), key=scores_main[i].__getitem__)
    closest_1_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind1])
    closest_1_score.append(scores_main[ind1])
    
    closest_2_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind2])
    closest_2_score.append(scores_main[ind2])
    
    closest_3_sfw_jobs.append(sfw_sector_acc['Job_Role_Replaced'][ind3])
    closest_3_score.append(scores_main[ind3])
    cf_jobs.append(acc_df['Job Title'][i])

df_acc = pd.DataFrame({'cf_jobs':cf_jobs, 'closest_1_sfw_jobs':closest_1_sfw_jobs, 'closest_1_score':closest_1_score, 'closest_2_sfw_jobs':closest_2_sfw_jobs, 'closest_2_score':closest_1_score, 'closest_3_sfw_jobs':closest_3_sfw_jobs, 'closest_3_score':closest_1_score})
df_acc.head(50)